In [ ]:
import pandas as pd

In [ ]:
# download and extract corpus
!wget https://github.com/OFAI/million-post-corpus/releases/download/v1.0.0/million_post_corpus.tar.bz2 && \
    tar -xf million_post_corpus.tar.bz2

In [ ]:
# convert sqlite data to csv files
!mkdir -p tmp/ && \
    cd tmp && \
    bash ../sqlite2csv.sh ../million_post_corpus/corpus.sqlite3

In [ ]:
# load csv tables into pandas
articles = pd.read_csv("tmp/Articles.csv")
posts = pd.read_csv("tmp/Posts.csv", dtype={"ID_Parent_Post": "O"})
annotations = pd.read_csv("tmp/Annotations_consolidated.csv")
folds = pd.read_csv("tmp/CrossValSplit.csv")

# merge tables into one
posts_labeled = pd.merge(pd.merge(posts, annotations, how="left"), folds, how="left").dropna(subset=["Category"])
posts_labeled.Value = posts_labeled.Value.astype(int)
posts_labeled.Fold = posts_labeled.Fold.astype(int)

In [ ]:
# write articles
articles.to_csv("articles.csv.xz", index=False, compression="xz")

In [ ]:
# write labeled posts
posts_labeled.to_csv("posts_labeled.csv.xz", index=False, compression="xz")

In [ ]:
# write unlabeled posts
annotated_ids = set(posts_labeled.ID_Post.unique())
posts[~posts.ID_Post.isin(annotated_ids)].to_csv("posts_unlabeled.csv.xz", index=False, compression="xz")